In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0,2"

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
tnts_san = SAN(TS=True, TN=True, depth=2)
tnts_san.compile(optimizer='adam', loss='mae')
tnts_san(np.random.random((2, 160, 36, 36, 3)));
tnts_san.summary()

Model: "san"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Sample 0 (Sequential)           │ (320, 18, 18, 64)      │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Sample 1 (Sequential)           │ (320, 9, 9, 32)        │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 0 (Sequential)            │ (320, 36, 36, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv1 1 (Sequential)            │ (320, 18, 18, 16)      │        11,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 0 (Sequential)            │ (320, 18, 18, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Conv2 1 (Sequential)            │ (320, 9, 9, 32)        │        13,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv1 0 (Sequential)        │ (320, 36, 36, 16)      │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv1 1 (Sequential)        │ (320, 18, 18, 8)       │         1,744 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv2 0 (Sequential)        │ (320, 18, 18, 32)      │        27,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ AttnConv2 1 (Sequential)        │ (320, 9, 9, 16)        │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask1 0 (Sequential)            │ (320, 36, 36, 1)       │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask1 1 (Sequential)            │ (320, 18, 18, 1)       │             9 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask2 0 (Sequential)            │ (320, 18, 18, 1)       │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Mask2 1 (Sequential)            │ (320, 9, 9, 1)         │            17 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Pooling & Downsample            │ (320, 18, 18, 32)      │             0 │
│ (Sequential)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (320, 8)               │         4,104 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Flatten & Dense (Sequential)    │ (320, 1)               │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147,597 (576.55 KB)

 Trainable params: 147,597 (576.55 KB)

 Non-trainable params: 0 (0.00 B)

In [3]:
batch_size = 128

rlap_tape = "/root/ssd_cache/rppg_training_data/rlap_160x36x36_all"

train_rlap = load_datatape(rlap_tape, fold='train', batch=batch_size)
valid_rlap = load_datatape(rlap_tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_rlap), KerasDataset(valid_rlap)

#training_set = training_set.apply_fn(motion_aug)
training_set = training_set.apply_fn(compress_aug)

rlap_160x36x36_all             datatape has been loaded.    24704 items total. fold=train selected
rlap_160x36x36_all             datatape has been loaded.     2944 items total. fold=val&extended_hr=False selected


In [4]:
#tnts_san.fit(training_set, validation_data=validation_set, epochs=20, callbacks=[keras.callbacks.ModelCheckpoint('../weights/tnts_san_36.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(tnts_san, training_set, validation_set, epochs=20, check_point_path='../weights/tnts_san_36.weights.h5')
tnts_san.load_weights('../weights/tnts_san_36.weights.h5')

Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.7188	Validation loss:0.4933	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.3299	Validation loss:  0.35	


Training:   0%|          | 0/193 [00:00<?, ?it/s]

Validating:   0%|          | 0/23 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.3288	Validation loss:0.3626	


In [5]:
eval_on_dataset(dataset_H5_mmpd, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_RLAP_MMPD.h5')
get_metrics('../results/TNTS-SAN-F2W_RLAP_MMPD.h5', dropped='False')

  0%|          | 0/660 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,10.635±0.184,18.894±3.149,12.606±0.223,0.4422
Whole,6.699±0.344,11.073±3.531,7.87±0.401,0.69739


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_RLAP_COHFACE.h5', fps=30)
get_metrics('../results/TNTS-SAN-F2W_RLAP_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,2.793±0.171,7.829±2.625,3.779±0.217,0.79008
Whole,2.013±0.328,4.657±2.83,2.676±0.405,0.91871


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_RLAP_PURE.h5')
get_metrics('../results/TNTS-SAN-F2W_RLAP_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,0.449±0.053,1.539±0.886,0.683±0.087,0.99776
Whole,0.244±0.082,0.673±0.563,0.367±0.146,0.9996


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(tnts_san), 160, (36, 36), step=4, batch=16, save='../results/TNTS-SAN-F2W_RLAP_UBFC.h5')
get_metrics('../results/TNTS-SAN-F2W_RLAP_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,1.209±0.089,2.365±1.104,1.238±0.095,0.99029
Whole,0.48±0.082,0.716±0.435,0.474±0.074,0.99922
